# Fine Tune Com Wikipédia



### Descriçao

---
1. O objetivo deste notebook é aprender sobre Fine Tune
2. Vamos descorrer sobre modelos de LLM e utilizar a biblioteca Torch para descorrer sobre assunto
3. Vamos ensinar nossa máquina com os dados do [Wikipédia](https://pt.wikipedia.org/wiki/Ci%C3%AAncia)
---

### Baixando os pacotes




In [14]:
!pip install transformers transformers[torch] datasets

### Importando as bibliotecas

In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import requests
from datasets import Dataset, load_dataset  # Biblioteca para carregar e manipular conjuntos de dados
import torch  # Biblioteca para operações de tensor e computação com GPU

# Importa o modelo pré-treinado GPT-2 e seu tokenizador
# DataCollatorForLanguageModeling é usado para preparar os dados para o treinamento
# Trainer e TrainingArguments auxiliam na configuração e treinamento do modelo

### importando o modelo

In [16]:
model_name = "pierreguillou/gpt2-small-portuguese"  # Nome do modelo pré-treinado em português

# Carregamos o tokenizador correspondente ao modelo
# O tokenizador é responsável por converter texto em tokens numéricos para entrada no modelo
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Carregamos o modelo GPT-2 pré-treinado para geração de texto
model = GPT2LMHeadModel.from_pretrained(model_name)

# Requisitando o Wikipedia

In [17]:
url = "https://pt.wikipedia.org/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy"
# URL da página da Wikipédia que queremos obter o texto
# Para fins de pesquisa, é melhor pegar uma única página em vez de usar a API completa do Wikipédia

response = requests.get(url)  # Faz uma requisição HTTP para obter o conteúdo da página
text_data = response.text  # Extrai o conteúdo da resposta em formato de texto

text_data  # Exibe o conteúdo bruto da página (HTML completo)

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="pt" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>The Hitchhiker\'s Guide to the Galaxy – Wikipédia, a enciclopédia livre</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vecto

In [18]:
# Divide o conteúdo da página em linhas usando a quebra de linha como delimitador
lines = text_data.split('\n')

# Cria um dicionário onde a chave "text" contém a lista de linhas extraídas
data_dict = {"text": lines}

# Converte o dicionário em um objeto Dataset da biblioteca `datasets`
# Isso permite manipular os dados de forma mais estruturada para uso no modelo
dataset = Dataset.from_dict(data_dict)

dataset  # Exibe o dataset criado

Dataset({
    features: ['text'],
    num_rows: 2107
})

#### Tokenização

In [19]:
# Função para tokenizar o dataset
def token_map(examples):
  return tokenizer(
    examples["text"],  # Tokeniza o texto do dataset
    padding="max_length",  # Preenche as sequências curtas até o comprimento máximo
    truncation=True,  # Trunca sequências que ultrapassam o limite
    max_length=128  # Define o comprimento máximo para cada sequência tokenizada
  )

# Aplica a função de tokenização ao dataset
# O parâmetro `batched=True` permite processar múltiplas amostras de uma vez, acelerando a tokenização
tokenized_dataset = dataset.map(token_map, batched=True)

Map:   0%|          | 0/2107 [00:00<?, ? examples/s]

In [20]:
# Exibe a estrutura do dataset tokenizado
print(tokenized_dataset)

# Visualiza um exemplo tokenizado
print(tokenized_dataset[0])

# Exibe as chaves disponíveis no dataset
print(tokenized_dataset.column_names)

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 2107
})
{'text': '<!DOCTYPE html>', 'input_ids': [28, 1, 36, 14617, 52, 57, 8215, 469, 84, 24883, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
['text', 'input_ids', 'attention_mask']


In [21]:
# Exibe o texto original da 400ª entrada no dataset tokenizado
tokenized_dataset[400]['text']

# Exibe os IDs dos tokens correspondentes ao texto original na 200ª entrada
# Cada ID representa um token no vocabulário do modelo
# tokenized_dataset[200]['input_ids']

# Exibe a máscara de atenção da 100ª entrada
# Indica quais tokens devem ser considerados pelo modelo (1) e quais são preenchimentos (0)
# tokenized_dataset[100]['attention_mask']

'\t\t\t\t\t<span>Vogons</span>'

In [22]:
# Preparando o lote de dados para o treinamento
# DataCollatorForLanguageModeling é um utilitário que agrupa os dados e os prepara para o modelo

data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer,  # Usa o tokenizador que criamos anteriormente
  mlm=False  # Define que não será usada a técnica de Masked Language Modeling (MLM)
  # MLM é usado em modelos como BERT, onde algumas palavras são ocultadas e o modelo precisa prever as palavras ausentes.
  # No caso do GPT-2, o treinamento é baseado em modelagem causal (prevendo a próxima palavra), então o MLM é desativado.
)

In [28]:
# Definição dos hiperparâmetros para o treinamento do modelo

training_args = TrainingArguments(
  output_dir="./output",  # Diretório onde os resultados do treinamento serão salvos
  overwrite_output_dir=True,  # Sobrescreve os arquivos existentes no diretório de saída
  num_train_epochs=1,  # Número de épocas (passagens completas pelo conjunto de dados)
  # Aqui está configurado para 1 época apenas para fins de demonstração

  per_device_train_batch_size=32,  # Tamanho do lote (batch size) por dispositivo (GPU/CPU)

  save_steps=10_000,  # Número de etapas entre cada salvamento dos checkpoints do modelo
  save_total_limit=2,  # Limita o número de checkpoints salvos para evitar consumo excessivo de armazenamento
  report_to="none"  # Evita que o Trainer tente enviar logs para o `wandb`
)

### criando o treinamento

In [29]:
# Inicializa o Trainer, que gerencia o processo de treinamento do modelo

trainer = Trainer(
  model=model,  # Modelo de linguagem que será treinado (GPT-2 pré-treinado)
  args=training_args,  # Hiperparâmetros definidos anteriormente
  data_collator=data_collator,  # Utilitário para preparar os lotes de dados
  train_dataset=tokenized_dataset  # Conjunto de dados tokenizado para treinamento
)

#### Treinando

In [30]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=66, training_loss=2.1147578846324575, metrics={'train_runtime': 85.4543, 'train_samples_per_second': 24.656, 'train_steps_per_second': 0.772, 'total_flos': 137635577856000.0, 'train_loss': 2.1147578846324575, 'epoch': 1.0})

### Salvando o modelo

In [31]:
# Salva o modelo treinado no diretório "meu_guia"
model.save_pretrained("meu_guia")

# Salva o tokenizador associado ao modelo no mesmo diretório
tokenizer.save_pretrained("meu_guia")

('meu_guia/tokenizer_config.json',
 'meu_guia/special_tokens_map.json',
 'meu_guia/vocab.json',
 'meu_guia/merges.txt',
 'meu_guia/added_tokens.json')

# Utilizando o Modelo

#importando a biblioteca que criamos

In [32]:
from transformers import pipeline

In [33]:
# Cria um pipeline para geração de texto usando o modelo treinado
gerador = pipeline("text-generation", model="meu_guia")

# Instrução para o modelo gerar um texto
instrucao = """Me fale sobre o Maycon"""

# Gera o texto com base na instrução fornecida
resultado = gerador(
  instrucao,
  truncation=True,  # Garante que o texto gerado não ultrapasse o limite de tokens
  max_length=600,  # Define o comprimento máximo do texto gerado
  do_sample=True,  # Ativa a amostragem estocástica para maior diversidade no texto
  temperature=0.9,  # Controla a criatividade do modelo (quanto maior, mais variado e imprevisível)
  top_k=50,  # Restringe a escolha das próximas palavras às 50 mais prováveis
  top_p=0.85  # Usa "nucleus sampling" para limitar a escolha de palavras ao grupo com probabilidade acumulada de 85%
)

Device set to use cuda:0


In [34]:
# Extrai o texto gerado do dicionário de resultados
resultado = resultado[0]['generated_text']

# Exibe o texto gerado pelo modelo
print(resultado)

Me fale sobre o Maycon e os outros dois "containerias".

A primeira, a "The Fountain", não possui nenhuma ligação com o enredo do filme, e está apenas em um dos livros. O romance tem um prólogo, "The Fountain" não é publicado na série. O filme foi escrito por George A. Romero e dirigido por David O. Selznick. O filme conta a história de um jovem de vinte anos chamado "Guiddy" que vive em um vilarejo perto de Chicago. A personagem principal, "Guiddy", é um garoto de dezesseis anos de idade que vive sob a tutela de um tio. A história foi escrita por Romero e Selznick e conta a história de um garoto que não gosta de ninguém, apenas de ser ignorado. Ele se encontra com seu tio "J.J." Jackson, um vendedor de sapatos que é o "Guide" de um loja de antiguidades em Chicago, onde ele tem de se mudar por todo o ano. Ele passa por uma sala de aula, onde seu tio o leva até o "Guide" de uma loja de antiguidades e o leva até o "Guide" de uma loja de sapatos, onde ele se apaixona por um homem da cidad

In [35]:
# Encontra o índice onde a instrução aparece no texto gerado
indice_fim_texto_original = resultado.find(instrucao)

# Remove a instrução do texto gerado, mantendo apenas o conteúdo novo
novo_texto = resultado[indice_fim_texto_original + len(instrucao):].strip().replace('\n\n', ' ')

# Exibe o texto final sem a instrução original
print(novo_texto)

e os outros dois "containerias". A primeira, a "The Fountain", não possui nenhuma ligação com o enredo do filme, e está apenas em um dos livros. O romance tem um prólogo, "The Fountain" não é publicado na série. O filme foi escrito por George A. Romero e dirigido por David O. Selznick. O filme conta a história de um jovem de vinte anos chamado "Guiddy" que vive em um vilarejo perto de Chicago. A personagem principal, "Guiddy", é um garoto de dezesseis anos de idade que vive sob a tutela de um tio. A história foi escrita por Romero e Selznick e conta a história de um garoto que não gosta de ninguém, apenas de ser ignorado. Ele se encontra com seu tio "J.J." Jackson, um vendedor de sapatos que é o "Guide" de um loja de antiguidades em Chicago, onde ele tem de se mudar por todo o ano. Ele passa por uma sala de aula, onde seu tio o leva até o "Guide" de uma loja de antiguidades e o leva até o "Guide" de uma loja de sapatos, onde ele se apaixona por um homem da cidade. A história de "Guiddy